In [5]:
import psycopg2, os, sys, torch
from transformers import AutoTokenizer, LongT5ForConditionalGeneration, LongT5Config, BatchEncoding
my_dir = sys.path[0]
sys.path.append(os.path.join(my_dir, 'src/data'))
sys.path.append(os.path.join(my_dir, 'src/models'))
from make_dataset import *
from train_model import *
from torch.utils.data import  DataLoader
from langchain.text_splitter import CharacterTextSplitter

In [10]:
from datasets import load_dataset
wiki = load_dataset("wikipedia", "20220301.en")

Downloading: 100%|██████████| 15.3k/15.3k [00:00<00:00, 344kB/s]


In [30]:
test = """solo ys khong"""
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 200,
    chunk_overlap  = 50,
    length_function = len,
)
texts = text_splitter.create_documents([test])
for i in texts:
    print(i.page_content)
    break

solo ys khong


In [8]:
valid_path = 'docs/ELI5_val.jsonl'
with open(valid_path, 'r') as json_file:
        json_list_valid = list(json_file)
valid_dataset = ELI5(json_list_valid, 'val')
bs = 1
valid_data_loader = DataLoader(valid_dataset, batch_size=bs,\
        shuffle=False)

tokenizer = AutoTokenizer.from_pretrained(
    "t5-small")
config = LongT5Config.from_pretrained("t5-small")
# model = LongT5ForConditionalGeneration.from_pretrained(
#     "model.pt", config=config)
model = torch.load('model.pt')

for i, batch in tqdm(enumerate(valid_data_loader)):
    if i < 10:
        continue    
    input_token = batch[0].input_ids.squeeze(0).to('cuda')
    input_mask = batch[0].attention_mask.squeeze(0).to('cuda')
    ans_token = batch[1]
#     print(tokenizer.decode(batch[0].input_ids, skip_special_tokens=True))
    output = model.module.generate(
            input_ids = input_token,
            attention_mask = input_mask,
            max_length=128,
            early_stopping=True,
            num_beams=5)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(tokenizer.decode(batch[0]['input_ids'].squeeze(0)[0]))
    print(ans_token)
    print(answer)
    break

You are using a model of type t5 to instantiate a model of type longt5. This is not supported for all configurations of models and can yield errors.
10it [00:29,  2.90s/it]

[CLS] Why is Google Fibre taking so long to roll out? [SEP] that the company announced the expansion of its beta service to cover 3 cities: Boston, Los Angeles, and Washington, DC, the speeds were still limited to up to 200 Mbit/s. In June 2016, Google Fiber acquired Webpass to boost its effort in its experiments with wireless technologies. As a result, Google Fiber put its effort on fiber to the premises on hold to explore more on the cheaper wireless alternative. By early 2017, the Webpass division of Google Fiber expanded 1 Gbit/s wireless service to customers in many cities in the United States. In November 2016, Atlas Networks, an ISP that serves -26.567433416333188 [SEP]</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [ ]:
# TODO: create database
def create_database():
    # TABLE 2 columns
    # 1 text, 1 embedding 
    commands = (
        """
        create table vendor (
            vendor_id serial primary key,
            vendor_name varchar(255) not null,
        )
        """,
        """
        create table parts (
            part_id serial primary key,
            part_name varchar(255) not null,
        )
        """,
        """
        create table part_drawings (
            part_id 
        )
        """
    )

In [32]:
def add_part(part_name):
    insert_part = "INSERT INTO parts(part_name) VALUES(%s) RETURNING part_id;"
conn = None
try:
    conn = psycopg2.connect(
        host='localhost',
        port="5432",
        dbname="suppliers",
        user="postgres",
        password=1)
    cur = conn.cursor()
    # cur.execute("")
    conn.commit()
    cur.close()
except psycopg2.DatabaseError as error:
    print(error)
finally:
    # print("Done!")
    if conn is not None:
        conn.close()